In [1]:
from pandas import DataFrame as pdDataFrame, read_json as pd_read_json
from common.config import *

analysis_config = get_analysis_config()
path = analysis_config['output_files']['eval']

eval_df = pd_read_json(path)

eval_df

,sdv.adult
fabric.regular,"{'sdv': {'score': 0.992828731244727, 'properti..."
holdout,"{'sdv': {'score': 0.9976801464742051, 'propert..."
sdv.tabular,"{'sdv': {'score': 0.85401852627936, 'propertie..."


In [2]:
for c in eval_df.columns:
    dataset_name = c.split('.', 1)[-1]
    eval_df[f'{c}.Score'] = eval_df[c].apply(lambda x: x.get('sdv', {}).get('score') if isinstance(x, dict) else x)
    for p in eval_df[c].iloc[0]['sdv']['properties'].keys():
        eval_df[f'{c}.{p}'] = eval_df[c].apply(lambda x: x.get('sdv', {}).get('properties', {}).get(p)  if isinstance(x, dict) else x)
    eval_df = eval_df.drop(columns=c)
eval_df = eval_df.T
eval_df.columns = ['holdout'] + [c for c in eval_df.columns if c != 'holdout']

In [3]:
eval_df_score = eval_df.T[[c for c in eval_df.T.columns if 'Score' in c]]
eval_df_score.rename(columns={c:c.rsplit('.', 1)[0] for c in eval_df_score.columns})
eval_df_score.columns = sorted(eval_df_score.columns)
eval_df_score

,sdv.adult.Score
holdout,0.992829
fabric.regular,0.997680
sdv.tabular,0.854019


In [ ]:
import json

metadata = {
    "outputs": [
        {
          "type": "web-app",
          "storage": "inline",
          "source": eval_df_score.to_html(),
        },
        {
          "type": "table",
          "storage": "inline",
          "format": "csv",
          "header": [" "] + list(eval_df_score.columns),
          "source": eval_df_score.to_csv()
        }
    ]
  }

with open("mlpipeline-ui-metadata.json", "w") as metadata_file:
    json.dump(metadata, metadata_file)